In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
data = load_iris()

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
y = data.target -1

In [5]:
y[y<0] = 0 

In [6]:
x_train,x_test,y_train,y_test = train_test_split(data.data,y,test_size = 0.25)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.transform(x_test)

In [8]:
class DeepClassifier():
    def __init__ (self,model_arch,no_examples):
        self.in_size = model_arch[0]
        self.out_size = model_arch[-1]
        DeepClassifier.m = no_examples
        self.hidden = model_arch[1:-1]
        param = {}
        DeepClassifier.model_arch = model_arch
        
        for l in range(1, len(model_arch)):
            param['W' + str(l)] = np.random.randn(DeepClassifier.model_arch[l], DeepClassifier.model_arch[l-1]) * 0.01
            param['b' + str(l)] = np.zeros((DeepClassifier.model_arch[l], 1))
        DeepClassifier.param = param
        DeepClassifier.losslist = []

    def linear_forward(A,w,b):
        z = np.dot(w,A) + b
        cache = (A,w,b)
        return z,cache
    
    def sigmoid(z):
        A = 1/(1+np.exp(-z))
        cache = z
        return A,cache
    
    def relu(z):
        A = np.maximum(0,z)
        cache = z
        return A,cache
        
    def linear_activation_forward(A_prev,w,b,activation):
        
        if activation == "sigmoid":
            Z,linear_cache  = DeepClassifier.linear_forward(A_prev,w,b)
            A,activation_cache  = DeepClassifier.sigmoid(Z)
            
        elif activation == "relu":
                Z, linear_cache = DeepClassifier.linear_forward(A_prev,w,b)
                A, activation_cache = DeepClassifier.relu(Z)

        cache = (linear_cache, activation_cache)
        return A, cache
    
    def L_model_forward(x,param):
        caches = []
        A = x
        
        for l in range(1,len(DeepClassifier.model_arch)-1):
            A_prev = A
            A, cache = DeepClassifier.linear_activation_forward(A_prev,DeepClassifier.param['W' + str(l)],DeepClassifier.param['b' + str(l)], activation = "relu")            
            caches.append(cache)
   
        AL, cache = DeepClassifier.linear_activation_forward(A, DeepClassifier.param['W' + str(len(DeepClassifier.model_arch)-1)], DeepClassifier.param['b' + str(len(DeepClassifier.model_arch)-1)], activation = "sigmoid")
        caches.append(cache)

        return AL, caches

    def compute_cost(AL, Y):
        
        cost = -1./DeepClassifier.m * np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
        cost = np.squeeze(cost)

        return cost

    def linear_backward(dZ, cache):
        A_prev, W, b = cache
        
        dW = 1./DeepClassifier.m * np.dot(dZ, A_prev.T)
        db = 1./DeepClassifier.m * np.sum(dZ, axis=1, keepdims=True)
        dA_prev = np.dot(W.T, dZ)

        return dA_prev, dW, db


    def sigmoid_backward(dA, cache):
   
        Z = cache 
        s = 1/(1+np.exp(-Z))
        dZ = dA * s * (1-s)
        
        return dZ

    def relu_backward(dA, cache):
    
        Z = cache
        dZ = np.array(dA, copy=True)  
        dZ[Z <= 0] = 0
        
        return dZ
    
    
    def linear_activation_backward(dA, cache, activation):
        linear_cache, activation_cache = cache
    
        if activation == "relu":
            dZ = DeepClassifier.relu_backward(dA, activation_cache)
            dA_prev, dW, db = DeepClassifier.linear_backward(dZ, linear_cache)

        elif activation == "sigmoid":
            dZ = DeepClassifier.sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = DeepClassifier.linear_backward(dZ, linear_cache)

        return dA_prev, dW, db
    
    def L_model_backward(AL, Y, caches):
        grads = {}
        
        L = len(DeepClassifier.model_arch)-1
        
        
        Y = Y.reshape(AL.shape)

        
        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

        current_cache = caches[L-1]
        grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = DeepClassifier.linear_activation_backward(dAL, current_cache, "sigmoid")
        # Loop from l=L-2 to l=0
        for l in reversed(range(L-1)):
            # lth layer: (RELU -> LINEAR) gradients.
            # Inputs: "grads["dA" + str(l + 1)], current_cache". 
            # Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
            current_cache = caches[l]
            dA_prev_temp, dW_temp, db_temp = DeepClassifier.linear_activation_backward(grads["dA"+str(l+1)], current_cache, "relu")
            grads["dA" + str(l)] = dA_prev_temp
            grads["dW" + str(l + 1)] = dW_temp
            grads["db" + str(l + 1)] = db_temp
        
        return grads

    def update_parameters(param, grads, learning_rate):
    # number of layers in the neural network
        L = len(DeepClassifier.model_arch)-1 
        # Update rule for each parameter
        for l in range(L):
            DeepClassifier.param["W" + str(l+1)] = DeepClassifier.param["W" + str(l+1)] - learning_rate*grads["dW" + str(l+1)]
            DeepClassifier.param["b" + str(l+1)] = DeepClassifier.param["b" + str(l+1)] - learning_rate*grads["db" + str(l+1)]
        return param


    def fit(self,x_train,y_train,epochs,learning_rate):
        
        for epoch in range(epochs):
                AL,caches = DeepClassifier.L_model_forward(x_train,DeepClassifier.param)
                cost = DeepClassifier.compute_cost(AL, y_train)
                self.losslist.append(cost)
                if epoch % 1000 == 0:
                    print(f"loss is {cost} in {epoch}th epoch ")
                grads = DeepClassifier.L_model_backward(AL, y_train, caches)
                self.param = DeepClassifier.update_parameters(DeepClassifier.param, grads, learning_rate)
                    

    def Predict(self,x):
        a = x
        for i in range(1,len(self.model_arch)-1):       
            z = np.dot(self.param["w" + str(i)].T,a) + self.param["b" + str(i)]
            a = np.tanh(z)
            
        z = np.dot(self.param["w" + str(len(self.model_arch)-1)].T,a) + self.param["b" + str(len(self.model_arch)-1)]
        a = np.tanh(z)
        return a  
                      
    def predict(self,x):
        A = x
        for l in range(1,len(DeepClassifier.model_arch)-1):
            A_prev = A
            A, cache = DeepClassifier.linear_activation_forward(A_prev,DeepClassifier.param['W' + str(l)],DeepClassifier.param['b' + str(l)], activation = "relu")            
            
        AL, cache = DeepClassifier.linear_activation_forward(A, DeepClassifier.param['W' + str(len(DeepClassifier.model_arch)-1)], DeepClassifier.param['b' + str(len(DeepClassifier.model_arch)-1)], activation = "sigmoid")
        

        return AL
        
        
        
        
        
        
                    
                    

In [9]:
Dcls = DeepClassifier([4,3,2,1],x_train_sc.shape[0])

In [10]:
Dcls.fit(x_train_sc.T,y_train,epochs=10000,learning_rate=0.1)

loss is 0.6931464060686929 in 0th epoch 
loss is 0.6463286217447384 in 1000th epoch 
loss is 0.6448581601143734 in 2000th epoch 
loss is 0.05958090759833543 in 3000th epoch 
loss is 0.04808079056524883 in 4000th epoch 
loss is 0.04559803984514632 in 5000th epoch 
loss is 0.04451964268265075 in 6000th epoch 
loss is 0.0439295958414382 in 7000th epoch 
loss is 0.0435680334607717 in 8000th epoch 
loss is 0.043318970217134885 in 9000th epoch 


In [14]:
y_pred = Dcls.predict(x_test_sc.T)

In [15]:
y_pred.shape

(1, 38)

In [13]:
y_test.shape

(38,)

In [17]:
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [18]:
y_pred

array([[0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
        0., 1., 0., 0., 0., 0.]])

In [21]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_pred.reshape(-1),y_test))
accuracy_score(y_test,y_pred.reshape(-1))

[[26  1]
 [ 1 10]]


0.9473684210526315